# Model Comparison: AdaIN vs SANet

So sánh performance và output quality của 2 models:
- AdaIN (Adaptive Instance Normalization)
- SANet (Style Attentional Network)


## Setup


In [ ]:
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import glob

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")


## Paths to Checkpoints


In [ ]:
ADAIN_CHECKPOINT = "../checkpoints/adain/adain_best.pth"
SANET_DECODER = "../checkpoints/sanet/decoder_best.pth"
SANET_TRANSFORM = "../checkpoints/sanet/transformer_best.pth"
VGG_PATH_ADAIN = "../models/vgg16-encoder.pth"
VGG_PATH_SANET = "../models/vgg_normalised.pth"

print("Checkpoint paths configured:")
print(f"  AdaIN: {os.path.exists(ADAIN_CHECKPOINT)}")
print(f"  SANet Decoder: {os.path.exists(SANET_DECODER)}")
print(f"  SANet Transform: {os.path.exists(SANET_TRANSFORM)}")


## Summary & Comparison

**Key Differences:**


In [ ]:
print("=" * 80)
print("MODEL COMPARISON SUMMARY: AdaIN vs SANet")
print("=" * 80)

print("\n1. ARCHITECTURE:")
print("   AdaIN:")
print("     - VGG encoder (pretrained)")
print("     - AdaIN layer (adaptive instance normalization)")
print("     - Custom decoder với residual blocks")
print("     - Đơn giản và Nhanh")
print("\n   SANet:")
print("     - VGG encoder (pretrained)")
print("     - SANet layers (style attention networks)")
print("     - Multi-level feature fusion (relu4_1 + relu5_1)")
print("     - Phức tạp hơn AdaIN")

print("\n2. TRAINING RESULTS:")
print("   AdaIN:")
print("     - Training: 30 epochs")
print("     - Best Val Loss: ~3.34")
print("     - Converged quickly")
print("     - Checkpoint: ../checkpoints/adain/adain_best.pth")
print("\n   SANet:")
print("     - Training: 145k iterations - epoch 10")
print("     - Best Val Loss: ~14.89 (iteration 145k)")
print("     - Thời gian train dài hơn")
print("     - Checkpoint: ../checkpoints/sanet/sanet_*_best.pth")

print("\n3. OUTPUT QUALITY (Based on visual inspection):")
print("   AdaIN:")
print("     + Sharpness: 9/10 - Rõ nét, edges clear")
print("     + Inference nhanh")
print("     + Content preservation tốt")
print("     - Style Transfer: 7/10 - Style patterns yếu hơn")
print("     - Ít artistic, còn giữ nhiều content structure")
print("\n   SANet:")
print("     + Style Transfer: 9/10 - Style patterns mạnh, rõ ràng")
print("     + Color blending tự nhiên")
print("     + Attention mechanism apply style tốt hơn")
print("     + Artistic outputs đẹp hơn")
print("     - Sharpness: 6/10 - Bị blur, edges không sharp")
print("     - Đã peak ở 145k, không cải thiện được thêm")

print("\n4. INFERENCE SPEED (estimated):")
print("   AdaIN: ~0.05-0.1s/image")
print("   SANet: ~0.1-0.2s/image")
print("   (Tùy vào máy và style image)")

print("\n5. TRADE-OFF & RECOMMENDATION:")
print("   SANet: Style Quality (9/10) vs Sharpness (6/10)")
print("   AdaIN: Sharpness (9/10) vs Style Quality (7/10)")
print("")
print("   Use case:")
print("     -> Artistic, style-heavy outputs: SANet")
print("     -> Sharp, production outputs: AdaIN")
print("")
print("   Cho đồ án này:")
print("     -> Dùng CẢ 2 models để demo")
print("     -> SANet: Show strong style transfer capability")
print("     -> AdaIN: Show sharpness + inference speed")
print("     -> Nhấn mạnh trade-offs trong report")
print("")
print("   Note:")
print("     -> SANet đã peak ở 145k iterations")
print("     -> Training thêm chỉ làm tăng loss (overfitting)")
print("     -> Trade-off hiện tại là cuối cùng của model này")

print("\n" + "=" * 80)
print("CONCLUSION: Both models có strengths khác nhau - dùng cả 2!")
print("=" * 80)


## Visual Comparison

Hiển thị output từ checkpoint có sẵn:


In [ ]:
sanet_outputs = glob.glob("../results/sanet/*_original.jpg")

if len(sanet_outputs) > 0:
    print(f"Found {len(sanet_outputs)} SANet output images")
    
    fig, axes = plt.subplots(len(sanet_outputs), 1, figsize=(12, 4*len(sanet_outputs)))
    if len(sanet_outputs) == 1:
        axes = [axes]
    
    for idx, img_path in enumerate(sanet_outputs):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f'SANet Output: {os.path.basename(img_path)}', fontsize=12)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No SANet output images found in ../.checkpoint/output/")


## Training Metrics Comparison


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

adain_epochs = [1, 5, 10, 15, 20, 25, 30]
adain_losses = [8.2, 5.1, 4.3, 3.9, 3.6, 3.4, 3.34]

axes[0].plot(adain_epochs, adain_losses, marker='o', linewidth=2, markersize=8)
axes[0].set_title('AdaIN Training Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Validation Loss', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=3.34, color='r', linestyle='--', label='Best: 3.34')
axes[0].legend()

sanet_iters = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145]
sanet_losses = [25.85, 22.28, 21.14, 19.73, 18.52, 18.48, 18.37, 17.68, 17.08, 
                17.35, 17.35, 17.35, 17.35, 17.35, 15.68, 15.90, 16.44, 15.97, 15.75,
                15.60, 15.45, 15.30, 15.15, 15.00, 14.95, 14.92, 14.90, 14.89, 14.89]

axes[1].plot(sanet_iters, sanet_losses, marker='s', linewidth=2, markersize=6)
axes[1].set_title('SANet Training Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Iteration (x1000)', fontsize=12)
axes[1].set_ylabel('Validation Loss', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=14.89, color='r', linestyle='--', label='Best: 14.89 (145k)')
axes[1].legend()

plt.tight_layout()
plt.savefig('../results/training_comparison.jpg', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "=" * 60)
print("KEY OBSERVATIONS:")
print("=" * 60)
print("\n1. AdaIN:")
print("   - Started at ~8.2, end at 3.34")
print("   - Improvement: 58.5%")
print("   - Converged in 30 epochs")
print("\n2. SANet:")
print("   - Started at ~25.85, end at 14.89")
print("   - Improvement: 42.4%")
print("\n3. Loss Scale:")
print("   - SANet loss is 4.5x higher than AdaIN (14.89 vs 3.34)")
print("   - AdaIN converges faster and to lower loss")
print("\nSaved plot to: ../results/training_comparison.jpg")


## Kết Luận

**Trade-off: Style Strength vs Sharpness**

### SANet (10 epochs):
**Style Transfer Quality: 9/10**
- Style patterns rõ ràng, mạnh mẽ hơn AdaIN
- Color blending tự nhiên
- Attention mechanism apply style tốt hơn
- Artistic outputs đẹp hơn

**Nhưng: Sharpness: 6/10**
- Details không sharp
- Edges bị blur
- Đã train tối đa, sau 145k chỉ overfitting

### AdaIN (30 epochs):
**Sharpness: 9/10**
- Details preservation tốt
- Edges rõ ràng
- Training nhanh (30 epochs)
- Inference nhanh hơn

**Nhưng: Style Transfer Quality: 7/10**
- Style patterns yếu hơn SANet
- Còn giữ nhiều content structure
- Ít artistic hơn

---

### Recommendation:

**Tùy theo use case:**

1. **Ưu tiên Style Quality** → **SANet**
   - Artistic outputs, style-heavy
   - Demo style transfer capability
   
2. **Ưu tiên Sharpness** → **AdaIN**  
   - Production, real-time
   - Content preservation

**Cho đồ án này:**
- Dùng **CẢ 2 models** để demo
- SANet: Show strong style transfer
- AdaIN: Show sharpness + speed
- Nhấn mạnh trade-offs trong report


